**Tecnológico de Monterrey**

**Maestría en Inteligencia Artificial Aplicada**

**ASIGNATURA: Proyecto Integrador**

**PROFESOR TITULAR: Dra. Grettel Barceló Alonso**

**ACTIVIDAD:Avance 4. Modelos Alternativos**

---
**EQUIPO 3**

*   Rayan Bahrein García Fabián - A01204055
*   Brandon Alexis del Ángel Gómez - A01795429
*   Fernando Jiménez Pereyra - A01734609

## Resumen ejecutivo

### Objetivo

Esta fase del proyecto tuvo como propósito evaluar diferentes algoritmos de aprendizaje supervisado aplicados al problema de predicción del contribution_score y su clasificación asociada, con el fin de identificar el modelo individual con mejor desempeño para su futura implementación.

### 1. Construcción de modelos alternativos

Se desarrollaron y evaluaron 14 modelos individuales, cubriendo tres enfoques:

* Regresión (predicción continua de contribution_score):

  * Algoritmos: LinearRegression, RandomForestRegressor, XGBRegressor, SVR, KNeighborsRegressor, DecisionTreeRegressor, GradientBoostingRegressor, LightGBMRegressor, MLPRegressor

* Clasificación Multiclase (Bajo / Medio / Alto):

  * Algoritmos: RandomForestClassifier, XGBoostClassifier

* Clasificación Binaria (Alto vs No Alto):

  * Algoritmos: LogisticRegression, RandomForestClassifier, XGBoostClassifier

Todos los modelos fueron construidos con pipelines de preprocesamiento y validados con GridSearchCV, registrando tanto métricas de desempeño como tiempos de entrenamiento.


### 2. Comparación de desempeño y selección

* Mejor modelo: RandomForestRegressor

  * MAE: 3.61 | R²: 0.80 | Tiempo: 2,576s

  * Otros modelos destacados: SVR, LightGBMRegressor, XGBRegressor

* Clasificación Multiclase:

  * Mejor modelo: XGBoostClassifier

  * Accuracy: 0.74 | F1 ponderado: 0.69 | Tiempo: 7s

  * RandomForestClassifier mostró desempeño competitivo pero con mayor tiempo (138s)

* Clasificación Binaria:

  * Mejor modelo: LogisticRegression

  * F1: 0.74 | AUC: 0.84 | Tiempo: 6.1s

  * Le sigue RandomForestClassifier con AUC más alto (0.847), pero mayor tiempo de entrenamiento (80s)


### 3. Justificación

Se priorizó la selección de modelos que balancean desempeño predictivo, tiempo de entrenamiento y posible interpretabilidad, considerando también la facilidad de integración futura en entornos productivos.

*    Para regresión, RandomForestRegressor fue elegido por su R² sobresaliente y estabilidad frente a no linealidades en los datos.

*    Para clasificación multiclase, XGBoostClassifier demostró una ventaja en precisión y tiempo de entrenamiento.

*    Para clasificación binaria, LogisticRegression ofrece una buena métrica F1, alto AUC, y es fácilmente interpretable.



## Importación de librerías y carga de datos

In [ ]:
import pandas as pd
import numpy as np
import time
import joblib

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

from xgboost import XGBClassifier

import xgboost as xgb
import lightgbm as lgb

El dataset con el que trabajamos fue generado en los pasos anteriores de procesamiento, limpieza e ingeniería de características.

In [ ]:
df = pd.read_csv('dataset_modelo_baseline.csv')

## Generación de etiquetas objetivo y separación de conjuntos

A partir de la variable continua `contribution_score` se generan dos etiquetas adicionales: una para clasificación multiclase (`score_label_multiclass`) y otra binaria (`bin_class`). Esto permite evaluar el rendimiento de distintos modelos según distintos enfoques: regresión, clasificación multiclase y binaria. Posteriormente, se separan las variables predictoras (`X`) de las variables objetivo para cada tipo de tarea supervisada.

In [ ]:
def clasificar_score(score):
    if score < 65:
        return 'Bajo'
    elif score < 80:
        return 'Medio'
    else:
        return 'Alto'

df['score_label_multiclass'] = df['contribution_score'].apply(clasificar_score)
# Crear variable binaria Alto vs No Alto (>=70)
df['bin_class'] = (df['contribution_score'] >= 70).astype(int)
# df['bin_class_label'] = df['bin_class'].map({1: 'Alto', 0: 'No Alto'})

In [ ]:
target_reg = 'contribution_score'
X = df.drop(columns=[target_reg, 'score_label_multiclass', 'bin_class', 'bin_class_label',
                     'project_id', 'employee_id', 'assignment_id'], errors='ignore')
y_reg = df[target_reg]
y_multi = df['score_label_multiclass']
y_bin = df['bin_class']

## División de datos para entrenamiento y prueba

Se realiza una división del conjunto de datos en entrenamiento y prueba, conservando la proporción de clases en la variable multiclase (`stratify=y_multi`). Esto es importante para evitar sesgos en el conjunto de validación y asegurar una evaluación representativa. El `random_state` garantiza reproducibilidad.

In [ ]:
X_train, X_test, yreg_train, yreg_test, ymulti_train, ymulti_test, ybin_train, ybin_test = train_test_split(
    X, y_reg, y_multi, y_bin,
    test_size=0.2, stratify=y_multi, random_state=42
)

## Procesamiento de variables numéricas y categóricas

Se identifican las variables categóricas y numéricas para aplicar transformaciones específicas en un `ColumnTransformer`. Las variables numéricas se estandarizan con `StandardScaler`, y las categóricas se codifican con `OneHotEncoder`, ignorando categorías desconocidas. Esta transformación asegura que todas las variables estén en un formato compatible para el entrenamiento de modelos, sin introducir sesgos por escalas o categorías.

In [ ]:
cat_cols = X.select_dtypes(include=['object']).columns
num_cols = X.select_dtypes(include=['int64','float64']).columns

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

# Ajustamos solo en X_train
X_train_proc = preprocessor.fit_transform(X_train)
X_test_proc  = preprocessor.transform(X_test)

## Definición de modelos de regresión y búsqueda de hiperparámetros

Se definen nueve modelos de regresión distintos utilizando algoritmos lineales, basados en árboles, redes neuronales, vecinos más cercanos y métodos de boosting. Cada modelo está acompañado de un conjunto de hiperparámetros para ser optimizados mediante `GridSearchCV`. Esto permite comparar tanto la precisión como la eficiencia de entrenamiento en una evaluación sistemática.

In [ ]:
model_configs_reg = {
    'LinearRegression': (
        LinearRegression(),
        {}  # no hiperparámetros para grid
    ),
    'RandomForestRegressor': (
        RandomForestRegressor(random_state=42),
        {'reg__n_estimators': [100, 200]}
    ),
    'XGBRegressor': (
        xgb.XGBRegressor(random_state=42),
        {'reg__n_estimators': [100, 200], 'reg__max_depth': [3, 6]}
    ),
    'KNeighborsRegressor': (
        KNeighborsRegressor(),
        {'reg__n_neighbors': [3, 5, 7]}
    ),
    'SVR': (
        SVR(),
        {'reg__C': [0.1, 1, 10], 'reg__kernel': ['rbf', 'linear']}
    ),
    'DecisionTreeRegressor': (
        DecisionTreeRegressor(random_state=42),
        {'reg__max_depth': [None, 10, 20]}
    ),
    'GradientBoostingRegressor': (
        GradientBoostingRegressor(random_state=42),
        {'reg__n_estimators': [100, 200], 'reg__learning_rate': [0.05, 0.1]}
    ),
    'LightGBMRegressor': (
        lgb.LGBMRegressor(random_state=42),
        {'reg__n_estimators': [100, 200], 'reg__num_leaves': [31, 50]}
    ),
    'MLPRegressor': (
        MLPRegressor(max_iter=500, random_state=42),
        {'reg__hidden_layer_sizes': [(50,), (100,)], 'reg__alpha': [0.0001, 0.001]}
    )
}

## Entrenamiento, ajuste y evaluación de cada modelo

Para cada modelo, se construye un pipeline que incluye el preprocesamiento y el estimador. Se aplica `GridSearchCV` para identificar la mejor combinación de hiperparámetros usando validación cruzada (3-fold) con la métrica de error absoluto medio (`MAE`). Se registra el tiempo total de entrenamiento y se evalúa el modelo final sobre el conjunto de prueba utilizando `MAE` y `R²`, métricas estándar para tareas de regresión. Los resultados se almacenan en una lista para su posterior comparación.

In [ ]:
results_reg = []
best_models_reg = {}

for name, (model, param_grid) in model_configs_reg.items():
    print(f"Entrenando regresor: {name}")
    pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('reg', model)
    ])
    grid = GridSearchCV(pipe, param_grid, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)
    start = time.time()
    grid.fit(X_train, yreg_train)
    elapsed = time.time() - start
    best = grid.best_estimator_
    y_pred = best.predict(X_test)
    mae = mean_absolute_error(yreg_test, y_pred)
    r2 = r2_score(yreg_test, y_pred)
    results_reg.append({
        'Model': name,
        'BestParams': grid.best_params_,
        'MAE': mae,
        'R2': r2,
        'TrainTime_s': elapsed
    })
    best_models_reg[name] = best
    # joblib.dump(best, f'best_regressor_{name}.joblib')
    print(f"{name}: MAE={mae:.3f}, R2={r2:.3f}, Time={elapsed:.1f}s")

# Guardar resultados en CSV
df_reg = pd.DataFrame(results_reg).sort_values('MAE')
# df_reg.to_csv('regression_comparison_results.csv', index=False)
# print("Resultados de regresión guardados en regression_comparison_results.csv")

Entrenando regresor: LinearRegression
LinearRegression: MAE=3.762, R2=0.709, Time=33.4s
Entrenando regresor: RandomForestRegressor
RandomForestRegressor: MAE=3.615, R2=0.801, Time=2577.0s
Entrenando regresor: XGBRegressor
XGBRegressor: MAE=4.391, R2=0.725, Time=5.3s
Entrenando regresor: KNeighborsRegressor
KNeighborsRegressor: MAE=5.815, R2=0.506, Time=40.6s
Entrenando regresor: SVR
SVR: MAE=3.937, R2=0.751, Time=3885.3s
Entrenando regresor: DecisionTreeRegressor
DecisionTreeRegressor: MAE=4.520, R2=0.656, Time=20.0s
Entrenando regresor: GradientBoostingRegressor
GradientBoostingRegressor: MAE=5.194, R2=0.635, Time=26.5s
Entrenando regresor: LightGBMRegressor
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 39.780577 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2889
[LightGBM] [Info] Number of data points in the train set: 16994, number

/home/alt9193/Documents/act4/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/alt9193/Documents/act4/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/alt9193/Documents/act4/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/alt9193/Documents/act4/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/alt9193/Documents/act4/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, 

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.032966 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4231
[LightGBM] [Info] Number of data points in the train set: 25491, number of used features: 1803
[LightGBM] [Info] Start training from score 68.734671
LightGBMRegressor: MAE=4.257, R2=0.737, Time=1637.0s
Entrenando regresor: MLPRegressor


/home/alt9193/Documents/act4/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


MLPRegressor: MAE=4.812, R2=0.632, Time=1402.7s


## Consolidación de resultados

Se consolidan los resultados de entrenamiento en un DataFrame ordenado por desempeño (MAE), lo cual permite identificar fácilmente los modelos más prometedores. Esta tabla es útil para documentar el rendimiento, hiperparámetros seleccionados y tiempo de entrenamiento, y servirá como base para seleccionar los dos mejores modelos para la etapa de ajuste fino.

In [ ]:
results_reg_df = pd.DataFrame(results_reg)
results_reg_df.sort_values(by='R2', ascending=False, inplace=True)

In [ ]:
results_reg_df

,Model,BestParams,MAE,R2,TrainTime_s
1,RandomForestRegressor,{'reg__n_estimators': 200},3.614991,0.800781,2576.957614
4,SVR,"{'reg__C': 10, 'reg__kernel': 'linear'}",3.936745,0.751142,3885.292482
7,LightGBMRegressor,"{'reg__n_estimators': 200, 'reg__num_leaves': 50}",4.257365,0.736723,1637.035079
2,XGBRegressor,"{'reg__max_depth': 6, 'reg__n_estimators': 200}",4.391310,0.724847,5.309225
0,LinearRegression,{},3.762273,0.709092,33.442690
5,DecisionTreeRegressor,{'reg__max_depth': None},4.520351,0.655922,19.986360
6,GradientBoostingRegressor,"{'reg__learning_rate': 0.1, 'reg__n_estimators...",5.193518,0.635306,26.459972
8,MLPRegressor,"{'reg__alpha': 0.001, 'reg__hidden_layer_sizes...",4.812238,0.632065,1402.680601
3,KNeighborsRegressor,{'reg__n_neighbors': 5},5.815198,0.506143,40.563136


Mostramos una comparación detallada entre distintos modelos de regresión en términos de error absoluto medio (MAE), coeficiente de determinación (R²) y tiempo de entrenamiento (`TrainTime_s`). El modelo RandomForestRegressor se posiciona como el de mejor rendimiento, obteniendo el MAE más bajo (3.61) y el R² más alto (0.80), lo cual indica una buena capacidad para explicar la variabilidad de la variable `contribution_score` con una precisión razonable. Sin embargo, esto viene acompañado de un costo computacional considerable (2576 segundos).

Modelos como SVR, LightGBMRegressor y XGBRegressor también obtienen resultados competitivos con R² por encima de 0.70, aunque difieren significativamente en el tiempo de entrenamiento: LightGBM muestra un buen balance entre desempeño (R² = 0.74) y eficiencia (1637 s), mientras que XGB se entrena mucho más rápido (5.3 s) aunque con un R² algo menor (0.72). Modelos como LinearRegression ofrecen rapidez (33 s) y una base razonable (R² = 0.70), pero su capacidad predictiva es superada por modelos no lineales. Finalmente, MLPRegressor, GradientBoostingRegressor y KNeighborsRegressor presentan desempeños más limitados, lo que sugiere que no capturan tan bien los patrones del problema en comparación con modelos basados en árboles más robustos.


## Entrenamiento de modelos para clasificación múltiple

Se define un diccionario de configuración para los modelos multiclase. En este caso, se utilizan dos clasificadores: `RandomForestClassifier` y `XGBClassifier`, junto con sus respectivos espacios de búsqueda de hiperparámetros para ajuste posterior mediante GridSearchCV

In [ ]:
model_configs_multi = {
    'RandomForest': (
        RandomForestClassifier(random_state=42),
        {'clf__n_estimators': [100, 200], 'clf__max_depth': [None, 10, 20]}
    ),
    'XGBoost': (
        XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42),
        {'clf__n_estimators': [100, 200], 'clf__max_depth': [3, 6]}
    )
}

In [ ]:
from sklearn.preprocessing import LabelEncoder

le_multi = LabelEncoder()
ymulti_train_encoded = le_multi.fit_transform(ymulti_train)
ymulti_test_encoded = le_multi.transform(ymulti_test)

Se inicializan dos litas vacías para almacenar los resultados de desempeño y los mejores modelos obtenidos durante la búsqueda.

Se crea un pipeline que incluye tanto el preprocesamiento como el modelo. Esto garntiza que todo el flujo de transformación se mantenga consistente durante la validación cruzada.

 Se entrena cada modelo utilizando búsqueda en rejilla (`GridSearchCV`) con validación cruzada de 3 folds. Se utiliza como métrica principal la `accuracy`. También se registra el tiempo de entrenamiento con `time.time()` para análisis posterior.

 Una vez entrenado, se recupera el mejor estimador según la búsqueda y se realizan predicciones obre el conjunto de prueba.

 Se evalúa el desempeño del modelo utilizando accuracy y F1 ponderado, lo cual es útil en clases desbalanceadas.



In [ ]:
results_multi = []
best_models_multi = {}

for name, (model, param_grid) in model_configs_multi.items():
    print(f"Entrenando clasificador multiclase: {name}")
    pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('clf', model)
    ])
    grid = GridSearchCV(pipe, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
    start = time.time()
    grid.fit(X_train, ymulti_train_encoded)
    elapsed = time.time() - start
    best = grid.best_estimator_
    y_pred = best.predict(X_test)
    acc = accuracy_score(ymulti_test_encoded, y_pred)
    f1 = f1_score(ymulti_test_encoded, y_pred, average='weighted')
    results_multi.append({
        'Model': name,
        'BestParams': grid.best_params_,
        'Accuracy': acc,
        'F1': f1,
        'TrainTime_s': elapsed
    })
    best_models_multi[name] = best
    print(f"{name}: Accuracy={acc:.3f}, F1={f1:.3f}, Time={elapsed:.1f}s")

# Guardar resultados en DataFrame
df_multi = pd.DataFrame(results_multi).sort_values('Accuracy', ascending=False)
print(df_multi)

Entrenando clasificador multiclase: RandomForest


RandomForest: Accuracy=0.724, F1=0.693, Time=138.1s
Entrenando clasificador multiclase: XGBoost


/home/alt9193/Documents/act4/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [12:40:22] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/alt9193/Documents/act4/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [12:40:22] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/alt9193/Documents/act4/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [12:40:22] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/alt9193/Documents/act4/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [12:40:22] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/hom

XGBoost: Accuracy=0.744, F1=0.690, Time=7.0s
          Model                                         BestParams  Accuracy  \
1       XGBoost    {'clf__max_depth': 3, 'clf__n_estimators': 100}  0.744077   
0  RandomForest  {'clf__max_depth': None, 'clf__n_estimators': ...  0.724463   

         F1  TrainTime_s  
1  0.689755     7.043153  
0  0.693244   138.119574  


In [ ]:
df_multi

,Model,BestParams,Accuracy,F1,TrainTime_s
1,XGBoost,"{'clf__max_depth': 3, 'clf__n_estimators': 100}",0.744077,0.689755,7.043153
0,RandomForest,"{'clf__max_depth': None, 'clf__n_estimators': ...",0.724463,0.693244,138.119574


*  Accuracy: XGBoost alcanza una mayor precisión general (74.4%) comparado con Random Forest (72.4%), lo que indica una mejor capacidad de clasificar correctamente las tres clases (Bajo, Medio, Alto).

*  F1 Score ponderado: Ambos modelos obtienen valores similares (~0.69), lo que sugiere un desempeño comparable cuando se consideran tanto precisión como recall, ponderado por la frecuencia de cada clase.

*  Eficiencia computacional: El modelo XGBoost es drásticamente más rápido, con un tiempo de entrenamiento de solo 7 segundos, frente a los 138 segundos del Random Forest. Esto puede ser crucial si se busca eficiencia para futuros reentrenamientos o despliegue.



Aunque ambos modelos muestran un rendimiento similar en cuanto a métricas, XGBoost se posiciona como la mejor opción en este caso al ofrecer una mayor precisión con un tiempo de entrenamiento más de 90% menor, lo que lo convierte en una alternativa eficiente y competitiva para tareas de clasificación multiclase en este proyecto.

## Clasificación binaria

Se definen tres modelos binarios (lineal y dos árboles) junto con sus respectivos hiperparámetros para optimizar con `GridSearchCV`

In [ ]:
model_configs_bin = {
    'LogisticRegression': (
        LogisticRegression(max_iter=1000, random_state=42),
        {'clf__C': [0.1, 1, 10]}
    ),
    'RandomForest': (
        RandomForestClassifier(random_state=42),
        {'clf__n_estimators': [100, 200], 'clf__max_depth': [None, 10, 20]}
    ),
    'XGBoost': (
        XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
        {'clf__n_estimators': [100, 200], 'clf__max_depth': [3, 6]}
    )
}

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss

Para cada modelo, se construye un pipeline que incluye el preprocesamiento (`ColumnTransformer`) y el modelo (`clf`). Se realiza búsqueda en rejilla (`GridSearchCV`) para optimizar los hiperparámetros en validación cruzada (cv=3). También se mide el tiempo de entrenamiento.

Una vez entrenado el mejor modelo, se generan predicciones sobre el conjunto de prueba (`X_test`). Se calculan métricas clave para clasificación binaria.

In [ ]:
results_bin = []
best_models_bin = {}

for name, (model, param_grid) in model_configs_bin.items():
    print(f"Entrenando clasificador binario: {name}")
    pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('clf', model)
    ])
    grid = GridSearchCV(pipe, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
    start = time.time()
    grid.fit(X_train, ybin_train)
    elapsed = time.time() - start
    best = grid.best_estimator_
    y_pred = best.predict(X_test)
    y_proba = best.predict_proba(X_test)[:, 1]
    acc = accuracy_score(ybin_test, y_pred)
    f1 = f1_score(ybin_test, y_pred)
    auc = roc_auc_score(ybin_test, y_proba)
    loss = log_loss(ybin_test, y_proba)
    results_bin.append({
        'Model': name,
        'BestParams': grid.best_params_,
        'Accuracy': acc,
        'F1': f1,
        'AUC': auc,
        'LogLoss': loss,
        'TrainTime_s': elapsed
    })
    best_models_bin[name] = best
    print(f"{name}: Accuracy={acc:.3f}, F1={f1:.3f}, AUC={auc:.3f}, LogLoss={loss:.3f}, Time={elapsed:.1f}s")

# Guardar resultados en DataFrame
df_bin = pd.DataFrame(results_bin).sort_values('Accuracy', ascending=False)
print(df_bin)

Entrenando clasificador binario: LogisticRegression
LogisticRegression: Accuracy=0.740, F1=0.740, AUC=0.844, LogLoss=0.478, Time=6.2s
Entrenando clasificador binario: RandomForest
RandomForest: Accuracy=0.734, F1=0.745, AUC=0.847, LogLoss=0.459, Time=80.3s
Entrenando clasificador binario: XGBoost


/home/alt9193/Documents/act4/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [12:43:56] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/alt9193/Documents/act4/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [12:43:56] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/alt9193/Documents/act4/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [12:43:56] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/alt9193/Documents/act4/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [12:43:56] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/hom

XGBoost: Accuracy=0.691, F1=0.747, AUC=0.825, LogLoss=0.495, Time=3.6s
                Model                                         BestParams  \
0  LogisticRegression                                     {'clf__C': 10}   
1        RandomForest  {'clf__max_depth': None, 'clf__n_estimators': ...   
2             XGBoost    {'clf__max_depth': 6, 'clf__n_estimators': 200}   

   Accuracy        F1       AUC   LogLoss  TrainTime_s  
0  0.739840  0.740044  0.843740  0.478348     6.153708  
1  0.734348  0.745069  0.847000  0.458986    80.254101  
2  0.690570  0.746725  0.824876  0.494897     3.566475  


In [ ]:
df_bin

,Model,BestParams,Accuracy,F1,AUC,LogLoss,TrainTime_s
0,LogisticRegression,{'clf__C': 10},0.739840,0.740044,0.843740,0.478348,6.153708
1,RandomForest,"{'clf__max_depth': None, 'clf__n_estimators': ...",0.734348,0.745069,0.847000,0.458986,80.254101
2,XGBoost,"{'clf__max_depth': 6, 'clf__n_estimators': 200}",0.690570,0.746725,0.824876,0.494897,3.566475


*    Logistic Regression logra el mayor valor de AUC (0.8437), lo cual es especialmente relevante en problemas con clases desbalanceadas, ya que mide la capacidad del modelo para separar las clases. También alcanza una buena accuracy (0.7398) y un F1 cercano (0.7400), lo que indica un desempeño balanceado entre precisión y recall. Además, su tiempo de entrenamiento es muy bajo (~6 segundos), lo cual lo hace eficiente.

*    Random Forest obtiene la mayor F1-score (0.7451), lo que sugiere un mejor equilibrio general entre precisión y recall que LogisticRegression, aunque con un costo computacional mayor (80 segundos). Además, presenta el mejor LogLoss (0.4589), lo cual indica buena calibración de probabilidades.

*    XGBoost tiene el mejor tiempo de entrenamiento (~3.5 segundos), pero su accuracy y AUC son los más bajos de los tres modelos. Aunque su F1 es comparable (0.7467), el LogLoss es el peor (0.4948), lo que indica menor calidad en la estimación de probabilidades.


Logistic Regression ofrece un excelente balance entre desempeño, estabilidad y eficiencia computacional, siendo el modelo más razonable. No obstante, si se da prioridad a la calidad del modelo en tareas críticas de clasificación (y hay tolerancia al tiempo de entrenamiento), Random Forest podría considerarse como candidato final, especialmente tras un ajuste adicional.